# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [3]:
# if run in Udacity WorkStation:
!pip -q install ./python
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.9 which is incompatible.


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [ ]:
# #if run in a local mac, drl python environment:
# from unityagents import UnityEnvironment
# import numpy as np
# env = UnityEnvironment(file_name="Tennis.app", no_graphics=True)

In [4]:
%reload_ext autoreload
%autoreload 2

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.         -6.65278625 -1.5        -0.          0.
  6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [7]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


### 4. Train agents with MADDPG

In [13]:
import torch
from unityagents import UnityEnvironment
import numpy as np
from nnmodels_tennis import Actor, Critic
from config_settings import Args
from replay_buffer import ReplayBuffer
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from maddpg_agents import MADDPG

agent = MADDPG(state_size, action_size, num_agents, 42) # state_size, action_size, n_agnets, random_seed
args=Args()

In [16]:
def maddpg(n_episodes=args.num_episodes, print_every=args.print_every): #000  #0   
    scores_window = deque(maxlen=args.print_every)         # save last 100 total scores in one episode
    all_scores = []
    avg_scores_window = []
    max_score = 0                                          # save best score in that run   
    solved = False
    best_avg=-9999
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        states = env_info.vector_observations              # get the current state  
        agent.reset()                                      # reset the agent                                     
        scores = np.zeros(num_agents)                      # initialize the score
        while True:
            actions = agent.act(states)                    # select an action from one agent
            env_info = env.step(actions)[brain_name]       # perform the action          
            next_states = env_info.vector_observations     # get next state
            rewards = env_info.rewards                     # get reward
            dones = env_info.local_done                    # check done
            agent.step(states, actions, rewards, next_states, dones, num_updates=args.num_updates) # agent step
            states = next_states
            scores += rewards
            if np.any(dones): break 
        
        avg_score = np.mean(scores) #score for one episode of mean of all agents
        scores_window.append(avg_score) # save last 100 avg_score scores  
        all_scores.append(avg_score)
        avg_scores_window.append(np.mean(scores_window))
        noise_damp = np.mean(scores_window)        
        print('\rEpisode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_window)), end="")
        
        if max_score < np.mean(scores_window): #save agent if 100 perfomance is better, that max_score
            torch.save(agent.actor_local.state_dict(), 'p3_checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'p3_checkpoint_critic.pth')
            max_score = np.mean(scores_window)   
            
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_window)))
            
            
        if max_score >= 0.5:
            if not solved:
                print('\nEnvironment solved in {:d} episodes!\t Score: {:.2f}'.format(i_episode, max_score))
                if max_score >=0.5 and max_score > best_avg: 
                    solved = True
                    best_avg = max_score
                    print('Exiting Traing!!') 
                    torch.save(agent.actor_local.state_dict(), 'p3_best_checkpoint_actor.pth')
                    torch.save(agent.critic_local.state_dict(), 'p3_best_checkpoint_critic.pth')
                    break
                    
    print("Training Done!")         
    return all_scores, avg_scores_window

all_scores, avg_scores_window = maddpg()

Episode 10	Average Score: -0.0050
Episode 20	Average Score: 0.00000
Episode 30	Average Score: 0.00000
Episode 40	Average Score: -0.0050
Episode 50	Average Score: 0.0300
Episode 60	Average Score: 0.0000
Episode 70	Average Score: -0.0050
Episode 80	Average Score: 0.00500
Episode 90	Average Score: -0.0050
Episode 100	Average Score: 0.0050
Episode 110	Average Score: 0.0050
Episode 120	Average Score: 0.0100
Episode 130	Average Score: 0.0000
Episode 140	Average Score: 0.00000
Episode 150	Average Score: -0.0050
Episode 160	Average Score: -0.0050
Episode 170	Average Score: 0.00000
Episode 180	Average Score: 0.0150
Episode 190	Average Score: -0.0050
Episode 200	Average Score: 0.00000
Episode 210	Average Score: 0.0050
Episode 220	Average Score: 0.0050
Episode 230	Average Score: 0.0100
Episode 240	Average Score: 0.0000
Episode 250	Average Score: -0.0050
Episode 260	Average Score: 0.00000
Episode 270	Average Score: 0.01000
Episode 280	Average Score: 0.0000
Episode 290	Average Score: -0.0050
Episod

KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(all_scores)+1), all_scores)
plt.plot(np.arange(1, len(avg_scores_window)+1), avg_scores_window)
plt.ylabel('Score')
plt.xlabel('Episode #')
ax.legend(['Score','Moving average'])
plt.show()

### 5. Evaluate the Trained Smart Multi-Agents

In [ ]:
agent = MADDPG(state_size, action_size, num_agents, 42) # state_size, action_size, n_agnets, random_seed
args=Args()

# load the weights from file
agent.actor_local.load_state_dict(torch.load('p3_best_checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('p3_best_checkpoint_critic.pth'))


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states)                        # select an action (for each agent)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
  
print("Score of trained agent playing a new game : {}".format(scores))
print("Mean {}".format(np.mean(scores)))


When finished, you can close the environment.

In [6]:
env.close()